In [ ]:
pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 15.1 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617752 sha256=06d7f601df6c54aa84d92783009ec58fefd070818407bbe2d1b060177c249b38
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
%matplotlib inline
from pyarrow import csv
from pathlib import Path

import heapq
from collections import defaultdict

import pandas as pd
import matplotlib.pylab as plt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
myfile = files.upload()

In [ ]:
filename = '/content/drive/MyDrive/01_음식물쓰레기_FOOD_WASTE.CSV'

In [ ]:
file2 = '/content/drive/MyDrive/02-1_내국인유동인구_KOREAN.CSV'

In [ ]:
file3 = '/content/drive/MyDrive/02-2_장기체류 외국인 유동인구_LONG_TERM_FRGN.CSV'

In [ ]:
file4 = '/content/drive/MyDrive/02-3_단기체류 외국인 유동인구_SHORT_TERM_FRGN.CSV'

In [ ]:
file5 = '/content/drive/MyDrive/03_거주인구_RESIDENT_POP.CSV'

In [ ]:
file6 = '/content/drive/MyDrive/04_음식관련 카드소비_CARD_SPENDING.CSV'

In [ ]:
food_df = pd.read_csv(filename,encoding='CP949',low_memory=False)
food_df

,base_date,city,emd_cd,emd_nm,em_area_cd,em_cnt,em_g,pay_amt
0,2018-01-01,제주시,50110670,외도동,W6X062,2,15500,464
1,2018-01-01,제주시,50110540,이도2동,W6X063,26,64650,1933
2,2018-01-01,제주시,50110630,아라동,W6X064,15,33500,1001
3,2018-01-01,제주시,50110610,삼양동,W6X065,4,8400,252
4,2018-01-01,제주시,50110540,이도2동,W6X066,21,51450,1538
...,...,...,...,...,...,...,...,...
2209856,2020-12-14,서귀포시,50130250,대정읍,W6Y722,16,26950,805
2209857,2020-12-14,서귀포시,50130250,대정읍,W6Y72A,3,17000,510
2209858,2020-12-14,서귀포시,50130310,안덕면,W6Y72B,5,5650,168
2209859,2020-12-14,서귀포시,50130310,안덕면,W6Y72C,11,20700,619


In [ ]:
korean_df = csv.read_csv(file2).to_pandas()

In [ ]:
korean_df

,base_date,time,city,emd_cd,emd_nm,sex,age,resd_pop_cnt,work_pop_cnt,visit_pop_cnt,resd
0,2018-01-01,1,b'\xc1\xa6\xc1\xd6\xbd\xc3',50110250,b'\xc7\xd1\xb8\xb2\xc0\xbe',b'\xbf\xa9\xbc\xba',0,638.5509,15.5400,151.5149,b'\xc1\xa6\xc1\xd6'
1,2018-01-01,1,b'\xc1\xa6\xc1\xd6\xbd\xc3',50110250,b'\xc7\xd1\xb8\xb2\xc0\xbe',b'\xbf\xa9\xbc\xba',0,0.0000,3.7249,171.3458,b'\xb1\xd7\xbf\xdc'
2,2018-01-01,1,b'\xc1\xa6\xc1\xd6\xbd\xc3',50110250,b'\xc7\xd1\xb8\xb2\xc0\xbe',b'\xbf\xa9\xbc\xba',10,668.4577,17.0308,166.0500,b'\xc1\xa6\xc1\xd6'
3,2018-01-01,1,b'\xc1\xa6\xc1\xd6\xbd\xc3',50110250,b'\xc7\xd1\xb8\xb2\xc0\xbe',b'\xbf\xa9\xbc\xba',10,0.0000,4.2577,195.8539,b'\xb1\xd7\xbf\xdc'
4,2018-01-01,1,b'\xc1\xa6\xc1\xd6\xbd\xc3',50110250,b'\xc7\xd1\xb8\xb2\xc0\xbe',b'\xbf\xa9\xbc\xba',20,609.7710,5.9490,148.7246,b'\xc1\xa6\xc1\xd6'
...,...,...,...,...,...,...,...,...,...,...,...
47250191,2021-06-30,24,b'\xbc\xad\xb1\xcd\xc6\xf7\xbd\xc3',50130620,b'\xbf\xb9\xb7\xa1\xb5\xbf',b'\xb3\xb2\xbc\xba',20,0.0000,0.0000,599.1969,b'\xb1\xd7\xbf\xdc'
47250192,2021-06-30,24,b'\xbc\xad\xb1\xcd\xc6\xf7\xbd\xc3',50130620,b'\xbf\xb9\xb7\xa1\xb5\xbf',b'\xb3\xb2\xbc\xba',20,187.5990,41.9998,75.5996,b'\xc1\xa6\xc1\xd6'
47250193,2021-06-30,24,b'\xbc\xad\xb1\xcd\xc6\xf7\xbd\xc3',50130620,b'\xbf\xb9\xb7\xa1\xb5\xbf',b'\xb3\xb2\xbc\xba',10,121.8467,0.0000,4.5128,b'\xc1\xa6\xc1\xd6'
47250194,2021-06-30,24,b'\xbc\xad\xb1\xcd\xc6\xf7\xbd\xc3',50130620,b'\xbf\xb9\xb7\xa1\xb5\xbf',b'\xb3\xb2\xbc\xba',80,122.2190,0.0000,4.5032,b'\xc1\xa6\xc1\xd6'


In [ ]:
long_df = pd.read_csv(file3,encoding='CP949',low_memory=False)

In [ ]:
short_df = pd.read_csv(file4,encoding='CP949',low_memory=False)

In [ ]:
resident_df = pd.read_csv(file5,low_memory=False,encoding='CP949')

In [ ]:
card_df = pd.read_csv(file6,low_memory=False,encoding='CP949')

In [ ]:
long_df

,base_date,time,nationality,city,emd_cd,emd_nm,resd_pop_cnt,work_pop_cnt,visit_pop_cnt,resd
0,2018-01-01,1,CAN,제주시,50110250,한림읍,6.3381,0.0000,4.5802,제주
1,2018-01-01,1,CHN,제주시,50110250,한림읍,442.1091,0.0000,5.7051,제주
2,2018-01-01,1,CHN,제주시,50110250,한림읍,0.0000,7.8695,51.3461,그외
3,2018-01-01,1,DEU,제주시,50110250,한림읍,0.0000,0.0000,8.2099,그외
4,2018-01-01,1,ETC,제주시,50110250,한림읍,15.9184,0.0000,2.8758,제주
...,...,...,...,...,...,...,...,...,...,...
23837312,2021-06-30,24,THA,서귀포시,50130620,예래동,0.0000,0.0000,3.3558,제주
23837313,2021-06-30,24,TWN,서귀포시,50130620,예래동,0.0000,0.0000,12.2655,그외
23837314,2021-06-30,24,USA,서귀포시,50130620,예래동,22.0624,0.0000,0.0000,제주
23837315,2021-06-30,24,USA,서귀포시,50130620,예래동,0.0000,0.0000,26.4749,그외


In [ ]:
short_df

,base_date,time,nationality,city,emd_cd,emd_nm,visit_pop_cnt
0,2018-01-01,1,CHN,제주시,50110250,한림읍,66.3170
1,2018-01-01,1,ETC,제주시,50110250,한림읍,21.1308
2,2018-01-01,1,HKG,제주시,50110250,한림읍,3.6771
3,2018-01-01,1,JPN,제주시,50110250,한림읍,5.5962
4,2018-01-01,1,USA,제주시,50110250,한림읍,3.9078
...,...,...,...,...,...,...,...
6059039,2021-06-30,24,USA,서귀포시,50130610,중문동,9.2120
6059040,2021-06-30,24,CHN,서귀포시,50130610,중문동,0.6039
6059041,2021-06-30,24,ETC,서귀포시,50130610,중문동,11.5473
6059042,2021-06-30,24,CHN,서귀포시,50130620,예래동,0.2013


In [ ]:
resident_df

,base_year,base_month,city,emd_cd,emd_nm,sex,resid_reg_pop,foreign_pop,total_pop
0,2018,1,제주시,50110590,건입동,남성,5085,146,5231
1,2018,1,제주시,50110590,건입동,여성,4715,82,4797
2,2018,1,제주시,50110256,구좌읍,남성,7965,368,8333
3,2018,1,제주시,50110256,구좌읍,여성,7609,184,7793
4,2018,1,서귀포시,50130253,남원읍,남성,9806,428,10234
...,...,...,...,...,...,...,...,...,...
3607,2021,6,제주시,50110250,한림읍,여성,10341,1140,11481
3608,2021,6,제주시,50110600,화북동,남성,12062,169,12231
3609,2021,6,제주시,50110600,화북동,여성,12238,161,12399
3610,2021,6,서귀포시,50130550,효돈동,남성,2660,39,2699


In [ ]:
resident_df

,base_year,base_month,city,emd_cd,emd_nm,sex,resid_reg_pop,foreign_pop,total_pop
0,2018,1,제주시,50110590,건입동,남성,5085,146,5231
1,2018,1,제주시,50110590,건입동,여성,4715,82,4797
2,2018,1,제주시,50110256,구좌읍,남성,7965,368,8333
3,2018,1,제주시,50110256,구좌읍,여성,7609,184,7793
4,2018,1,서귀포시,50130253,남원읍,남성,9806,428,10234
...,...,...,...,...,...,...,...,...,...
3607,2021,6,제주시,50110250,한림읍,여성,10341,1140,11481
3608,2021,6,제주시,50110600,화북동,남성,12062,169,12231
3609,2021,6,제주시,50110600,화북동,여성,12238,161,12399
3610,2021,6,서귀포시,50130550,효돈동,남성,2660,39,2699


In [ ]:
card_df

In [ ]:
resident_df.shape
resident_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3612 entries, 0 to 3611
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   base_year      3612 non-null   int64 
 1   base_month     3612 non-null   int64 
 2   city           3612 non-null   object
 3   emd_cd         3612 non-null   int64 
 4   emd_nm         3612 non-null   object
 5   sex            3612 non-null   object
 6   resid_reg_pop  3612 non-null   int64 
 7   foreign_pop    3612 non-null   int64 
 8   total_pop      3612 non-null   int64 
dtypes: int64(6), object(3)
memory usage: 254.1+ KB


In [ ]:
resident_df.drop(['date'],axis=1)

,base_year,base_month,city,emd_cd,emd_nm,sex,resid_reg_pop,foreign_pop,total_pop
0,2018,1,제주시,50110590,건입동,남성,5085,146,5231
1,2018,1,제주시,50110590,건입동,여성,4715,82,4797
2,2018,1,제주시,50110256,구좌읍,남성,7965,368,8333
3,2018,1,제주시,50110256,구좌읍,여성,7609,184,7793
4,2018,1,서귀포시,50130253,남원읍,남성,9806,428,10234
...,...,...,...,...,...,...,...,...,...
3607,2021,6,제주시,50110250,한림읍,여성,10341,1140,11481
3608,2021,6,제주시,50110600,화북동,남성,12062,169,12231
3609,2021,6,제주시,50110600,화북동,여성,12238,161,12399
3610,2021,6,서귀포시,50130550,효돈동,남성,2660,39,2699


In [ ]:
resident_df['base_month'] = resident_df['base_month'].map(str(0)+str(resident_df['base_month']) 

ValueError: ignored

In [ ]:
from datetime import datetime
def find_datetime(row):
  year = row['base_year']
  month = row['base_month']
  s = f"{year} {month} 1"
  d = datetime.strptime(s,"%Y %m %d")
  d.strftime("%Y-%m-%d")

In [ ]:
resident_df['date'] = resident_df.apply(find_datetime,axis=1)
resident_df['date'] = pd.to_datetime(resident_df['date']) #datetime 형식으로 바꿔줌
resident_df['date'].head(5)

0   NaT
1   NaT
2   NaT
3   NaT
4   NaT
Name: date, dtype: datetime64[ns]